In [1]:
import os

In [2]:
train_cats_dir = "./Data1/cats_and_dogs_small/train/cats"
train_dogs_dir = "./Data1/cats_and_dogs_small/train/dogs"
test_cats_dir = "./Data1/cats_and_dogs_small/test/cats"
test_dogs_dir = "./Data1/cats_and_dogs_small/test/dogs"
validation_cats_dir = "./Data1/cats_and_dogs_small/validation/cats"
validation_dogs_dir = "./Data1/cats_and_dogs_small/validation/dogs"

print("train(개) 개수 : ",len(os.listdir(train_dogs_dir)))
print("train(고양이) 개수 : ",len(os.listdir(train_cats_dir)))
print("test(개) 개수 : ",len(os.listdir(test_dogs_dir)))
print("test(고양이) 개수 : ",len(os.listdir(test_cats_dir)))
print("validation(개) 개수 : ",len(os.listdir(validation_dogs_dir)))
print("validation(고양이) 개수 : ",len(os.listdir(validation_cats_dir)))

train(개) 개수 :  1000
train(고양이) 개수 :  1000
test(개) 개수 :  500
test(고양이) 개수 :  500
validation(개) 개수 :  500
validation(고양이) 개수 :  500


### generator를 이용한 이미지 전처리

In [ ]:
# 이미지 전처리 (전처리(변환), 증식)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 픽셀값을 0-255 범위에서 0.0-1.0 범위로 변경
train_gen = ImageDataGenerator(rescale=1.0/255)
test_gen = ImageDataGenerator(rescale=1.0/255)

# 폴더로부터 데이터를 가져와서 처리
train_dir = "./Data1/cats_and_dogs_small/train/"
test_dir = "./Data1/cats_and_dogs_small/test/"
validation_dir = "./Data1/cats_and_dogs_small/validation/"

train_generator = train_gen.flow_from_directory()

test_generator = test_gen.flow_from_directory()